In [19]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

import sys
import time
import datetime 
from dateutil.parser import parse
import pytz

import urllib2
from bs4 import BeautifulSoup

from twitter import TwitterHTTPError
from oauth_setup import oauth_login

t = oauth_login()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
tweetlist = ['Uno...', '...due...', 'Tre.. Prova.']

for line in tweetlist: 
    t.statuses.update(status=line)
    print line
    print '...'
    time.sleep(15) # Sleep for 15 seconds

print "All done!"

Uno...
...
...due...
...
Tre.. Prova.
...
All done!


In [13]:
#t.statuses.home_timeline()

In [168]:
t.statuses.update(status="""
San Francisco Bay Entrance (Golden Gate)
YYYY-MM-DD
Sun   HH:MM-HH:MM 
Flood HH:MM-HH:MM (max x.y knots)
Ebb   HH:MM-HH:MM (max x.y knots)
""")

{u'contributors': None,
 u'coordinates': None,
 u'created_at': u'Fri Jun 16 01:01:08 +0000 2017',
 u'entities': {u'hashtags': [],
  u'symbols': [],
  u'urls': [],
  u'user_mentions': []},
 u'favorite_count': 0,
 u'favorited': False,
 u'geo': None,
 u'id': 875518570980974592,
 u'id_str': u'875518570980974592',
 u'in_reply_to_screen_name': None,
 u'in_reply_to_status_id': None,
 u'in_reply_to_status_id_str': None,
 u'in_reply_to_user_id': None,
 u'in_reply_to_user_id_str': None,
 u'is_quote_status': False,
 u'lang': u'in',
 u'place': None,
 u'retweet_count': 0,
 u'retweeted': False,
 u'source': u'<a href="https://www.nodc.noaa.gov/dsdt/cwtg/cpac.html" rel="nofollow">nereus</a>',
 u'text': u'San Francisco Bay Entrance (Golden Gate)\nYYYY-MM-DD\nSun   HH:MM-HH:MM \nFlood HH:MM-HH:MM (max x.y knots)\nEbb   HH:MM-HH:MM (max x.y knots)',
 u'truncated': False,
 u'user': {u'contributors_enabled': False,
  u'created_at': u'Fri Jul 17 14:35:16 +0000 2009',
  u'default_profile': False,
  u'default

In [ ]:
'http://tides.mobilegeographics.com/zones/'
'http://tides.mobilegeographics.com/zones/:Europe/'
'http://tides.mobilegeographics.com/zones/:Europe/Gibraltar'

'http://tides.mobilegeographics.com/index.html'

In [40]:
url='https://tidesandcurrents.noaa.gov/stationhome.html?id=9414290#sensors'
flob = urllib2.urlopen(url)
s = flob.read()
flob.close()
soup = BeautifulSoup(s)

In [68]:
#dat = soup.findAll('div', {'class':'row-fluid'})
#for i in range(len(dat)):
#for sss in soup.findAll('div', {'id':'attext'}):

dat[11].findAll('div', {'id': 'wltext'})

[<div id="wltext"></div>]

In [69]:
url='http://tides.mobilegeographics.com/locations/5545.html'

flob = urllib2.urlopen(url)
s = flob.read()
flob.close()
soup = BeautifulSoup(s)
print soup.findAll('pre', {'class':'predictions-table'})[0].get_text()

San Francisco Bay Entrance (Golden Gate), California Current
37.8167° N, 122.4833° W

2017-06-15 Thu  3:40 PM PDT    3.0 knots  Max Flood
2017-06-15 Thu  7:14 PM PDT   -0.0 knots  Slack, Ebb Begins
2017-06-15 Thu  8:33 PM PDT   Sunset
2017-06-15 Thu  9:37 PM PDT   -2.3 knots  Max Ebb
2017-06-16 Fri  1:11 AM PDT    0.0 knots  Slack, Flood Begins
2017-06-16 Fri  3:46 AM PDT    1.6 knots  Max Flood
2017-06-16 Fri  5:47 AM PDT   Sunrise
2017-06-16 Fri  6:19 AM PDT   -0.0 knots  Slack, Ebb Begins
2017-06-16 Fri  9:45 AM PDT   -3.3 knots  Max Ebb
2017-06-16 Fri  1:17 PM PDT    0.0 knots  Slack, Flood Begins
2017-06-16 Fri  4:31 PM PDT    2.9 knots  Max Flood
2017-06-16 Fri  8:00 PM PDT   -0.0 knots  Slack, Ebb Begins
2017-06-16 Fri  8:34 PM PDT   Sunset
2017-06-16 Fri 10:33 PM PDT   -2.6 knots  Max Ebb
2017-06-17 Sat  2:24 AM PDT    0.0 knots  Slack, Flood Begins
2017-06-17 Sat  4:35 AM PDT   Last Quarter
2017-06-17 Sat  4:53 AM PDT    1.7 knots  Max Flood
2017-06-17 Sat  5:47 AM PDT   Sunri

In [171]:
s = tide_table[5]

20

In [167]:
tide_table = soup.findAll('pre', {'class':'predictions-table'})[0].get_text().split('\n')
location = tide_table[0]
def parse_tides(s):
    tokens = s.split('PDT')
    ts = parse(tokens[0]).replace(tzinfo=pytz.timezone('US/Pacific'))
    conditions = tokens[1].split('knots')
    
    try:
        knots = float(conditions[0].replace(' ', ''))
        conditions = conditions[1].replace(' ', '').replace(',', '')
    except ValueError:
        knots = np.NaN
        conditions = 'Sunrise' if ts.hour<12 else 'Sunset'
    return({
        'time':ts,
        'knots':knots,
        'conditions':conditions
     })

tides = pd.DataFrame([parse_tides(tide) for tide in tide_table[3:-1]])
tides = tides.set_index('time')
tides.index = pd.DatetimeIndex(tides.index)


In [142]:

def format_tides_info(tides, day = str(datetime.date.today() + datetime.timedelta(hours=24))):
    tides[tides.conditions!='Sunset'][day:day]
    sun_times = tides[tides.conditions=='Sunset'][day:day].index.strftime('%H:%M').tolist()
    """{0}: Sun rises at {1}, sets at {2}""".format(day, sun_times[0], sun_times[1])
    
    
    Flood begin:end (Max)
        
    OR
    
    Flood until end (Max)
    
    Ebb   begin:end (Max)
        

datetime.date(2017, 6, 16)

In [144]:
San Francisco Bay Entrance (Golden Gate)
YYYY-MM-DD
Sun   HH:MM-HH:MM 
Flood HH:MM-HH:MM (max x.y knots)
        
OR

Flood until HH:MM (max x.y knots)

Ebb   HH:MM-HH:MM (max x.y knots)

,conditions,knots
time,,
2017-06-16 02:04:00-07:00,SlackFloodBegins,0.0
2017-06-16 04:39:00-07:00,MaxFlood,1.6
2017-06-16 07:12:00-07:00,SlackEbbBegins,-0.0
2017-06-16 10:38:00-07:00,MaxEbb,-3.3
2017-06-16 14:10:00-07:00,SlackFloodBegins,0.0
2017-06-16 17:24:00-07:00,MaxFlood,2.9
2017-06-16 20:53:00-07:00,SlackEbbBegins,-0.0
2017-06-16 23:26:00-07:00,MaxEbb,-2.6


'2017-06-16: Sun rises at 06:40, sets at 21:27'

In [104]:
tide_table[3:8]

[u'2017-06-15 Thu  3:40 PM PDT    3.0 knots  Max Flood',
 u'2017-06-15 Thu  7:14 PM PDT   -0.0 knots  Slack, Ebb Begins',
 u'2017-06-15 Thu  8:33 PM PDT   Sunset',
 u'2017-06-15 Thu  9:37 PM PDT   -2.3 knots  Max Ebb',
 u'2017-06-16 Fri  1:11 AM PDT    0.0 knots  Slack, Flood Begins']

In [110]:
tide_table[4].split('PDT')#[1].split('  ')

[u'2017-06-15 Thu  7:14 PM ', u'   -0.0 knots  Slack, Ebb Begins']

In [118]:
float(tide_table[4].split('PDT')[1].split('knots')[0].replace(' ', ''))

-0.0

datetime.datetime(2017, 6, 15, 15, 40, tzinfo=<DstTzInfo 'US/Pacific' LMT-1 day, 16:07:00 STD>)

In [93]:
time

u'2017-06-15 Thu  3:40 PM PDT'